In [1]:
import os
import sys
from pathlib import Path

mod_path = os.path.join(Path.cwd().parent.parent)
if mod_path not in sys.path:
    sys.path.append(mod_path)

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold

from src.data import *
from src.features.utils import *
from src.model.tree_based import ModelXgBoost

In [3]:
# data with shape 28007, 33 [transaction related features]
train = pd.read_csv('../../data/processed/train.csv')

In [4]:
train

,Unnamed: 0,ID,Deposit,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,...,mean_amt_paid,median_amt_paid,max_amt_paid,min_amt_paid,stddev_amt_paid,b1,b2,b3,b4,b5
0,0,ID_MR53LEX,2500,0.0,DAILY,55,3,Male,NaN,Coast Region,...,517.26,350.0,3600.0,55.0,652.49,770.0,280.0,1320.0,1200.0,660.0
1,1,ID_3D7NQUH,2500,0.0,DAILY,55,3,Male,26.0,South Rift,...,737.87,655.0,2940.0,380.0,452.03,770.0,655.0,660.0,660.0,605.0
2,2,ID_0IWQNPI,2400,0.0,DAILY,50,3,Male,21.0,Mount Kenya Region,...,1126.67,980.0,2850.0,200.0,1005.32,250.0,200.0,610.0,1350.0,1500.0
3,3,ID_IY8SYB9,2000,0.0,DAILY,40,7,Female,26.0,Mount Kenya Region,...,1126.00,1140.0,2200.0,380.0,511.30,380.0,600.0,1100.0,1300.0,780.0
4,4,ID_9XHL7VZ,2000,0.0,DAILY,40,7,Male,27.0,North Rift,...,353.59,190.0,2640.0,40.0,559.24,520.0,80.0,40.0,40.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28002,28002,ID_CDZ24L9,2500,0.0,DAILY,55,3,Male,NaN,Coast Region,...,1753.21,1635.0,5235.0,800.0,1069.37,1750.0,1600.0,1500.0,800.0,1670.0
28003,28003,ID_0XINELS,2400,0.0,DAILY,50,3,Female,NaN,Nairobi Region,...,1166.25,1025.0,2980.0,450.0,786.66,550.0,1200.0,1250.0,850.0,1050.0
28004,28004,ID_PAU9JJU,2000,0.0,DAILY,40,3,Male,29.0,South Rift,...,952.22,1020.0,2580.0,200.0,695.70,1020.0,1110.0,1080.0,660.0,200.0
28005,28005,ID_K866QHS,2000,0.0,DAILY,40,7,Female,57.0,Mount Kenya Region,...,1241.25,1180.0,2000.0,970.0,323.04,970.0,1240.0,1200.0,1000.0,1120.0


In [5]:
train.drop(columns = 'Unnamed: 0', inplace=True)
train.columns

Index(['ID', 'Deposit', 'AccessoryRate', 'rateTypeEntity', 'RatePerUnit',
       'DaysOnDeposit', 'MainApplicantGender', 'Age', 'Region', 'Occupation',
       'Term', 'TotalContractValue', 'ExpectedTermDate', 'FirstPaymentDate',
       'LastPaymentDate', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6',
       'SplitPaymentsHistory', 'nb_payments', 'amount_paid',
       'percent_amt_paid', 'mean_amt_paid', 'median_amt_paid', 'max_amt_paid',
       'min_amt_paid', 'stddev_amt_paid', 'b1', 'b2', 'b3', 'b4', 'b5'],
      dtype='object')

In [6]:
train.shape

(28007, 35)

In [7]:
# Region has certain NaN values which might cause issues while encoding
# As total NaNs constitute ~5% of the data (1446) we remove it as of now
print(train['Region'].isna().sum() / train.shape[0] * 100)
train.dropna(subset=['Region'], how='all', inplace=True)

5.162994965544328


In [8]:
train.shape

(26561, 35)

In [9]:
# split_payment_history_df = train[["ID", "SplitPaymentsHistory"]]
id_arr = train[["ID"]]

target = train[['m1', 'm2', 'm3', 'm4', 'm5', 'm6']]
train_arr = train.drop(columns=['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 
                                'ID',
                                'SplitPaymentsHistory',
                                'ExpectedTermDate', 
                                'FirstPaymentDate',
                                'LastPaymentDate',])

## Approach 1

### Model train on initial hp

In [10]:
def approach_one_model(x_train, y_train, x_test):
    model = ModelXgBoost(train_array=x_train, train_target=y_train)
    model.train_model()  # Default h.params (Checkout the code)
    predict = model.trained_model.predict(x_test)
    
    return model, predict

In [11]:
def encode_and_drop(full_array, data_type, tr_encoder=None):
#     print(tr_encoder)
    categorical_array = full_array[full_array.select_dtypes(exclude=['number']).columns]
    numerical_array = full_array.drop(columns=X_train.select_dtypes(exclude=['number']).columns)
#     print(numerical_array.shape)
    encoded_array, encoder = one_hot_encoding(
        categorical_frame=categorical_array, 
        type_of_data=data_type,
        fitted_encoder=tr_encoder,
        conv=True
    )
#     print(encoded_array.shape)
    final_array = pd.concat([numerical_array.reset_index(drop=True), 
                             encoded_array.reset_index(drop=True)], axis=1)
    final_array.index = numerical_array.index
#     print(final_array.shape)
    return numerical_array, encoded_array, encoder

In [12]:
# split data into train and test sets
from sklearn.model_selection import train_test_split

seed = 10
X_train, X_test, y_train, y_test = train_test_split(train_arr, target, test_size=0.45, random_state=seed)

In [13]:
model_dict = dict()
predict_dict = dict()
for model_no, target_col in enumerate(["m1", "m2", "m3", "m4", "m5", "m6"]):
    og_frame, encoded_train, encoder_model = encode_and_drop(X_train, "train", None)
    og_frame_test, encoded_test, encoder_model = encode_and_drop(X_test, "test", encoder_model)
    model_dict[f"M_{model_no}"], predict_dict[f"m{model_no+1}_pred"] = approach_one_model(
        x_train=encoded_train,
        y_train=y_train[[target_col]],
        x_test=encoded_test
    )    

### Calculation of RMSE

In [14]:
pred_frame = pd.DataFrame(predict_dict)
pred_frame.index = X_test.index

In [15]:
full_test_array = pd.concat([X_test, y_test, pred_frame], axis=1)

In [16]:
full_test_array = pd.merge(full_test_array, id_arr, how='left', left_index=True, right_index=True)

In [17]:
sub_file = SubmissionFile(
    validation_data=full_test_array,
    type_of_data='validation'
).execute()

In [18]:
sub_file.shape

(71718, 3)

In [19]:
sub_file['SquaredError'] = np.square(sub_file['Target'] - sub_file['Prediction'])

In [20]:
rmse = np.sqrt(np.sum(sub_file['SquaredError'])/sub_file.shape[0])
print('Final RMSE --> ', rmse)

Final RMSE -->  876.9076296273787
